In [1]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from hnmchallenge.dataset import Dataset
from hnmchallenge.filtered_dataset import FilterdDataset
from hnmchallenge.stratified_dataset import StratifiedDataset
from hnmchallenge.constant import *
import datetime

In [78]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

In [79]:
dataset = StratifiedDataset()
dr = DataReader()

In [80]:
# creating H&M features in holdin 
feature = dataset.get_holdin()

In [81]:
item_feature=feature.drop(["sales_channel_id","price"], axis=1)

In [82]:
item_feature

,t_dat,customer_id,article_id
0,2018-09-20,0,0
1,2018-09-20,0,0
2,2019-08-05,0,0
3,2019-08-16,0,644
4,2019-10-06,0,1952
...,...,...,...
10955395,2020-09-22,1136204,17478
10955396,2020-09-22,1136205,10033
10955397,2020-09-22,1136205,7580
10955398,2020-09-22,1136205,1765


In [83]:
item_feature1=item_feature[item_feature["t_dat"]>="2020-09-01"]

In [84]:
item_feature1.reset_index()

,index,t_dat,customer_id,article_id
0,457,2020-09-14,19,4705
1,512,2020-09-01,23,2873
2,513,2020-09-01,23,16951
3,514,2020-09-01,23,6749
4,515,2020-09-01,23,14886
...,...,...,...,...
174687,10955395,2020-09-22,1136204,17478
174688,10955396,2020-09-22,1136205,10033
174689,10955397,2020-09-22,1136205,7580
174690,10955398,2020-09-22,1136205,1765


In [85]:
item_feature1

,t_dat,customer_id,article_id
457,2020-09-14,19,4705
512,2020-09-01,23,2873
513,2020-09-01,23,16951
514,2020-09-01,23,6749
515,2020-09-01,23,14886
...,...,...,...
10955395,2020-09-22,1136204,17478
10955396,2020-09-22,1136205,10033
10955397,2020-09-22,1136205,7580
10955398,2020-09-22,1136205,1765


In [58]:
item_feature1=item_feature1.drop("index",axis=1)

KeyError: "['index'] not found in axis"

In [86]:
item_feature1

,t_dat,customer_id,article_id
457,2020-09-14,19,4705
512,2020-09-01,23,2873
513,2020-09-01,23,16951
514,2020-09-01,23,6749
515,2020-09-01,23,14886
...,...,...,...
10955395,2020-09-22,1136204,17478
10955396,2020-09-22,1136205,10033
10955397,2020-09-22,1136205,7580
10955398,2020-09-22,1136205,1765


In [87]:
duplicated_rows = item_feature[item_feature.duplicated(subset=[DEFAULT_USER_COL, DEFAULT_ITEM_COL])]
count_mb = duplicated_rows.groupby(DEFAULT_ITEM_COL).count()
count_mb = count_mb.reset_index()[[DEFAULT_ITEM_COL, "t_dat"]].rename(
    columns={"t_dat": "count"}
)

In [88]:
item_feature=pd.merge(count_mb,item_feature["article_id"], on="article_id",how="outer")

In [89]:
item_feature["count"]=item_feature["count"].fillna(0)

In [90]:
item_feature["article_id"].drop_duplicates()

0               0
3301            1
5387            2
6470            3
8346            4
            ...  
10955395    21552
10955396    22065
10955397    22066
10955398    20576
10955399    21879
Name: article_id, Length: 21131, dtype: int64

In [72]:
duplicated_rows = item_feature1[item_feature1.duplicated(subset=[DEFAULT_USER_COL, DEFAULT_ITEM_COL])]
count_mb1 = duplicated_rows.groupby(DEFAULT_ITEM_COL).count()
count_mb1 = count_mb1.reset_index()[[DEFAULT_ITEM_COL, "t_dat"]].rename(
    columns={"t_dat": "count_1st_sept"}
)

In [73]:
item_feature1=pd.merge(count_mb1,item_feature1["article_id"], on="article_id",how="outer")

In [75]:
item_feature1["count_1st_sept"]=item_feature1["count_1st_sept"].fillna(0)

In [35]:
item_features=count_mb.join(count_mb1.set_index('article_id'), on='article_id')

In [36]:
item_features['count_1st_sept']=item_features['count_1st_sept'].fillna(0)

In [37]:
item_features

,article_id,count,count_1st_sept
0,0,3300,0.0
1,1,2085,0.0
2,2,1082,0.0
3,3,1875,0.0
4,4,5343,8.0
...,...,...,...
20389,21987,1,1.0
20390,21993,3,3.0
20391,21997,2,2.0
20392,22001,3,3.0


In [ ]:
dr = DataReader()
feature.to_feather(
        dr.get_preprocessed_data_path() / "stratified_holdin_item_dataset.feather"
    )